import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.functions import when

create local SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/06 18:52:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


read csv with inferschema

In [3]:
data = spark.read.csv("ds_salaries.csv", inferSchema=True, header=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [6]:
data = spark.read.csv("ds_salaries.csv", inferSchema=True, header=True)

write schema of scv on screen

In [12]:
# data.schema
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [7]:
schema = StructType() \
      .add("", IntegerType(), False) \
      .add("work_year", IntegerType(), False) \
      .add("experience_level", StringType(), False) \
      .add("employment_type", StringType(), False) \
      .add("job_title", StringType(), False) \
      .add("salary", IntegerType(), False) \
      .add("salary_currency", StringType(), False) \
      .add("salary_in_usd", IntegerType(), False) \
      .add("employee_residence", StringType(), False) \
      .add("remote_ratio", IntegerType(), False) \
      .add("company_location", StringType(), False) \
      .add("company_size", StringType(), False)

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [8]:
data = spark.read.csv('ds_salaries.csv', header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [9]:
data.printSchema()

root
 |-- : integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [10]:
data.show(5)

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|   |work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|              GB|

print data in dataframe using display(df.toPandas())

In [11]:
display(data.toPandas())

,,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [12]:
df_job_title = data.select('job_title').distinct().collect()
df_job_title

[Row(job_title='3D Computer Vision Researcher'),
 Row(job_title='Lead Data Engineer'),
 Row(job_title='Head of Machine Learning'),
 Row(job_title='Data Specialist'),
 Row(job_title='Data Analytics Lead'),
 Row(job_title='Machine Learning Scientist'),
 Row(job_title='Lead Data Analyst'),
 Row(job_title='Data Engineering Manager'),
 Row(job_title='Staff Data Scientist'),
 Row(job_title='ETL Developer'),
 Row(job_title='Director of Data Engineering'),
 Row(job_title='Product Data Analyst'),
 Row(job_title='Principal Data Scientist'),
 Row(job_title='AI Scientist'),
 Row(job_title='Director of Data Science'),
 Row(job_title='Machine Learning Engineer'),
 Row(job_title='Lead Data Scientist'),
 Row(job_title='Machine Learning Infrastructure Engineer'),
 Row(job_title='Data Science Engineer'),
 Row(job_title='Machine Learning Manager'),
 Row(job_title='Research Scientist'),
 Row(job_title='Head of Data'),
 Row(job_title='Cloud Data Engineer'),
 Row(job_title='Machine Learning Developer'),
 Ro

print all rows from df_job_titles without truncating jobs

In [13]:
df_job_title

[Row(job_title='3D Computer Vision Researcher'),
 Row(job_title='Lead Data Engineer'),
 Row(job_title='Head of Machine Learning'),
 Row(job_title='Data Specialist'),
 Row(job_title='Data Analytics Lead'),
 Row(job_title='Machine Learning Scientist'),
 Row(job_title='Lead Data Analyst'),
 Row(job_title='Data Engineering Manager'),
 Row(job_title='Staff Data Scientist'),
 Row(job_title='ETL Developer'),
 Row(job_title='Director of Data Engineering'),
 Row(job_title='Product Data Analyst'),
 Row(job_title='Principal Data Scientist'),
 Row(job_title='AI Scientist'),
 Row(job_title='Director of Data Science'),
 Row(job_title='Machine Learning Engineer'),
 Row(job_title='Lead Data Scientist'),
 Row(job_title='Machine Learning Infrastructure Engineer'),
 Row(job_title='Data Science Engineer'),
 Row(job_title='Machine Learning Manager'),
 Row(job_title='Research Scientist'),
 Row(job_title='Head of Data'),
 Row(job_title='Cloud Data Engineer'),
 Row(job_title='Machine Learning Developer'),
 Ro

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [14]:
df_analytic = data.groupBy("job_title").agg(
    avg("salary_in_usd").alias("avg_salary"),
    min("salary_in_usd").alias("min_salary"),
    max("salary_in_usd").alias("max_salary")
)

print all rows from df_analytic without trancating jobs

In [15]:
df_analytic.show(n=df_analytic.count(), truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [25]:
# ordered = Window.orderBy(df_analytic.avg_salary.desc())
# df_analytic = df_analytic.withColumn('row_id', row_number().over(ordered))

df_analytic = df_analytic.orderBy(df_analytic.avg_salary.desc()) \
                         .withColumn('row_id', monotonically_increasing_id())

print all data from df_analytic

In [26]:
df_analytic.show(n=df_analytic.count(), truncate=False)

+----------------------------------------+------------------+----------+----------+------+
|job_title                               |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------------+------------------+----------+----------+------+
|Data Analytics Lead                     |405000.0          |405000    |405000    |0     |
|Principal Data Engineer                 |328333.3333333333 |185000    |600000    |1     |
|Financial Data Analyst                  |275000.0          |100000    |450000    |2     |
|Principal Data Scientist                |215242.42857142858|148261    |416000    |3     |
|Director of Data Science                |195074.0          |130026    |325000    |4     |
|Data Architect                          |177873.9090909091 |90700     |266400    |5     |
|Applied Data Scientist                  |175655.0          |54238     |380000    |6     |
|Analytics Engineer                      |175000.0          |135000    |205300    |7     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [27]:
columns = df_analytic.columns
columns.remove('row_id')
columns.insert(0, 'row_id')
df_analytic = df_analytic.select(columns)

print df_analytic now

In [28]:
df_analytic.show()

+------+--------------------+------------------+----------+----------+
|row_id|           job_title|        avg_salary|min_salary|max_salary|
+------+--------------------+------------------+----------+----------+
|     0| Data Analytics Lead|          405000.0|    405000|    405000|
|     1|Principal Data En...| 328333.3333333333|    185000|    600000|
|     2|Financial Data An...|          275000.0|    100000|    450000|
|     3|Principal Data Sc...|215242.42857142858|    148261|    416000|
|     4|Director of Data ...|          195074.0|    130026|    325000|
|     5|      Data Architect| 177873.9090909091|     90700|    266400|
|     6|Applied Data Scie...|          175655.0|     54238|    380000|
|     7|  Analytics Engineer|          175000.0|    135000|    205300|
|     8|     Data Specialist|          165000.0|    165000|    165000|
|     9|        Head of Data|          160162.6|     32974|    235000|
|    10|Machine Learning ...|          158412.5|     12000|    260000|
|    1

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [29]:
partitions = Window.partitionBy(data.experience_level)
df_exp_lvl = data.withColumn('biggest_salary', max(data.salary_in_usd).over(partitions))

print here df_exp_lvl

In [30]:
df_exp_lvl.show(n=df_exp_lvl.count(), truncate=False)

+---+---------+----------------+---------------+----------------------------------------+--------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|   |work_year|experience_level|employment_type|job_title                               |salary  |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+----------------------------------------+--------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|5  |2020     |EN              |FT             |Data Analyst                            |72000   |USD            |72000        |US                |100         |US              |L           |250000        |
|10 |2020     |EN              |FT             |Data Scientist                          |45000   |EUR            |51321        |FR                |0           |FR              

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [31]:
df_best = df_exp_lvl.filter(df_exp_lvl.salary == df_exp_lvl.biggest_salary) \
                    .select(["", "experience_level", "biggest_salary", "employee_residence"])

print df_best

In [32]:
df_best.show()

+---+----------------+--------------+------------------+
|   |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 21|              MI|        450000|                IN|
| 33|              MI|        450000|                US|
| 97|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [33]:
df_best = df_best.drop_duplicates(["experience_level"])

print df_best

In [34]:
df_best.show()

+---+----------------+--------------+------------------+
|   |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 21|              MI|        450000|                IN|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [35]:
df_new_best = df_best.select(["experience_level", "biggest_salary", "employee_residence"]) \
    .withColumn("experience_level", \
    when(df_best.experience_level.eqNullSafe("MI"), "middle") \
    .when(df_best.experience_level.eqNullSafe("SE"), "senior") \
    .otherwise(None))

print df_new_best

In [36]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        250000|                US|
|            null|        600000|                US|
|          middle|        450000|                IN|
|          senior|        412000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [38]:
df_new_best.write.csv("1.csv", header=True)
df_final = spark.read.csv("1.csv", header=True, inferSchema=True)

print df_final

In [39]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        250000|                US|
|            null|        600000|                US|
|          middle|        450000|                IN|
|          senior|        412000|                US|
+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [40]:
df_final = df_final.dropna(subset=["experience_level"])

print df_final

In [41]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|          middle|        450000|                IN|
|          senior|        412000|                US|
+----------------+--------------+------------------+



last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [42]:
salary_in_usd = df_final.agg(max("biggest_salary")).first()[0]
salary_in_usd

450000

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system